In [3]:
from warnings import warn

import numpy as np
import scipy as sp
from numpy import asarray
from scipy.sparse import (issparse,
                          SparseEfficiencyWarning, csc_matrix, csr_matrix, lil_matrix)
from scipy.linalg import LinAlgError
import copy
from tqdm import tqdm

def spinv_triangular(A):
    """
    Invert sparse lower triangular matrix A.

    Parameters
    ----------
    A : (M, M) sparse matrix
        A sparse square lower triangular matrix. Should be in CSR format.
    
    Returns
    -------
    x : (M, M) sparse matrix

    Raises
    ------
    LinAlgError
        If `A` is singular or not triangular.
    ValueError
        If shape of `A` does not match the requirements.

    Notes
    -----
    .. versionadded:: 0.19.0

    Examples
    --------
    >>> import numpy as np
    >>> from scipy.sparse import csr_matrix
    >>> from scipy.sparse.linalg import spsolve_triangular
    >>> A = csr_matrix([[3, 0, 0], [1, -1, 0], [2, 0, 1]], dtype=float)
    >>> Ainv = spinv_triangular(A, B)
    """


    # Check the input for correct type and format.
    if not (issparse(A) and A.format == "csr"):
        warn('CSR matrix format is required. Converting to CSR matrix.',
             SparseEfficiencyWarning, stacklevel=2)
        A = csr_matrix(A)
    else:
        A = A.copy()
    
    n = A.shape[0]
    if n != A.shape[1]:
        raise ValueError(
            f'A must be a square matrix but its shape is {A.shape}.')

    # sum duplicates for non-canonical format [???]
    A.sum_duplicates()

    # Init x as (a copy of) A.
    x = lil_matrix(sp.sparse.eye(n))

    row_indices = range(n)

    # Fill x iteratively.
    for i in tqdm(row_indices):

        # Get indices for i-th row.
        indptr_start = A.indptr[i]
        indptr_stop = A.indptr[i + 1]

        A_diagonal_index_row_i = indptr_stop - 1
        A_off_diagonal_indices_row_i = slice(indptr_start, indptr_stop - 1)
       
        # Incorporate off-diagonal entries.
        A_column_indices_in_row_i = A.indices[A_off_diagonal_indices_row_i]
        A_values_in_row_i = A.data[A_off_diagonal_indices_row_i]
        x[i] -= x[A_column_indices_in_row_i].T @ A_values_in_row_i

        # Compute i-th entry of x.
        x[i] /= A.data[A_diagonal_index_row_i]

    return csr_matrix(x)

In [5]:
from scipy.sparse import eye
A = csr_matrix([[3, 0, 0], [1, -1, 0], [2, 0, 1]], dtype=float)
Ainv = spinv_triangular(A)
lu = splu(A)
Ainv2 = spsolve(A, eye(3))

100%|██████████████████████████████████████████████████████| 3/3 [00:00<00:00, 485.90it/s]


In [7]:
print(Ainv)
print(Ainv2)

  (0, 0)	0.3333333333333333
  (1, 0)	0.3333333333333333
  (1, 1)	-1.0
  (2, 0)	-0.6666666666666666
  (2, 2)	1.0
[[ 0.33333333  0.          0.        ]
 [ 0.33333333 -1.         -0.        ]
 [-0.66666667  0.          1.        ]]


In [11]:
Ainv2 = lu.solve_sparse(eye(3))
print(Ainv2)

  (0, 0)	0.3333333333333333
  (1, 0)	0.3333333333333333
  (2, 0)	-0.6666666666666666
  (1, 1)	-1.0
  (2, 2)	1.0


In [19]:
A=lil_matrix(Ainv2)

In [55]:
from scipy.sparse.linalg import inv
print(inv(A))

  (0, 0)	0.3333333333333333
  (1, 0)	0.3333333333333333
  (1, 1)	-1.0
  (2, 0)	-0.6666666666666666
  (2, 2)	1.0


In [27]:
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import diags, triu, vstack, hstack, csr_matrix, eye
from scipy.sparse.linalg import spsolve_triangular
from tqdm import tqdm
from scikits.umfpack import splu

def process_sparse_matrix(sparse_matrix):
    ploidy = sparse_matrix.data.max()
    sparse_matrix_csc = sparse_matrix.tocsc()

    # Step 1: Adjust allele flips
    flip_alleles = []
    for i in range(sparse_matrix_csc.shape[1]):
        column = sparse_matrix_csc[:, i].toarray()
        mean_value = column.mean()
        flip_alleles.append(mean_value > ploidy / 2)
        if mean_value > ploidy / 2:
            modified_column = ploidy - column
            sparse_matrix_csc[:, i] = sparse.csc_matrix(modified_column)

    # Step 2: Construct Xh
    Xh = None
    for n in range(1, int(ploidy) + 1):
        X_carrier = sparse_matrix_csc >= n
        X_carrier = X_carrier.astype(int)
        R_carrier = X_carrier.transpose().dot(X_carrier)
        
        temp = R_carrier.copy()
        for i in range(R_carrier.shape[0]):
            row_indices = R_carrier.indices[R_carrier.indptr[i]:R_carrier.indptr[i+1]]
            row_data = R_carrier.data[R_carrier.indptr[i]:R_carrier.indptr[i+1]]
            temp.data[temp.indptr[i]:temp.indptr[i+1]] = (row_data >= R_carrier[i, i])
        if n == 1:
            Xh = temp
        else:
            Xh = Xh.multiply(temp)
    Xh = Xh.tocsc()
    Xh.eliminate_zeros()

    # Step 3: Break ties in Xh
    ties = Xh.multiply(Xh.transpose())
    Xh = Xh - triu(ties, k=1)
    Xh.eliminate_zeros()

    # Step 4: Reorder Xh
    row_counts = np.diff(Xh.indptr)
    sorted_indices = np.argsort(-row_counts)
    Xh_reordered = Xh[sorted_indices, :][:, sorted_indices]

    # Step 5: Solve the triangular system
    n = Xh.shape[0]
    Xh_reordered = Xh_reordered.astype(int).tocsr()
    b = eye(n)
    #A = spinv_triangular(Xh_reordered)
    lu = splu(Xh_reordered)
    A = lu.solve_sparse(csc_matrix(b))

    # Undo the reordering
    original_order = np.argsort(sorted_indices)
    Ahaplo = A[original_order, :][:, original_order]
    Asample = sparse_matrix_csc @ Ahaplo
    Ahaplo = eye(n) - Ahaplo

    # Step 7: Construct Ainit
    m, n = Asample.shape
    zeros_matrix = csr_matrix((m+n, m))
    vertical_stack = vstack([Asample, Ahaplo])
    Ainit = hstack([zeros_matrix, vertical_stack])

    return Ainit, flip_alleles

# You would call this function with your sparse_matrix as follows:
# Ainit, flip_alleles = process_sparse_matrix(sparse_matrix)


In [28]:
from scipy.io import mmread
X = mmread('/Users/loconnor/Dropbox/linearArg/linearg_shared/genotypes.mtx')

In [ ]:
%%time
Ainit, flip_alleles = process_sparse_matrix(X)

In [60]:
from scipy.io import mmwrite
mmwrite('/Users/loconnor/Dropbox/linearArg/data/linearg_python.mtx',Ainit)


In [61]:
X

<5008x51461 sparse matrix of type '<class 'numpy.float64'>'
	with 1419782 stored elements in COOrdinate format>

In [62]:
Ainit

<56469x56469 sparse matrix of type '<class 'numpy.float64'>'
	with 569507 stored elements in COOrdinate format>